In [1]:
import numpy as np, matplotlib.pyplot as plt
import MDAnalysis as mda
from CG_cluster import *

# Please cite this paper if you think this notebook is useful for your research! Thank you very much!
# https://pubs.acs.org/doi/10.1021/acs.jctc.3c01053
# Jiangbo Wu, Weizhi Xue, and Gregory A. Voth
# K-Means Clustering Coarse-Graining (KMC-CG): A Next Generation Methodology for Determining Optimal Coarse-Grained Mappings of Large Biomolecules
# J. Chem. Theory Comput. 2023

In [2]:
#####start editing#####
system_info = mda.Universe("md.pdb", "md.trr")
#load your structure file and trajectory here
#####end editing#####
protein = system_info.select_atoms("protein") # select protein atoms

## Chain A/B means Arp 2/3

## Chain A/B

In [3]:
##### generate cgmapping for chain A/B
for segid in 'AB': # do kmc-cg for each chain A/B
    ca_atoms = system_info.atoms.select_atoms("segid %s and name CA" % segid) # select CA atoms
    frames = [] # initialize the coordinates of CA atoms
    for frame in system_info.trajectory[-700:]: # select the last 700 frames for clustering
        frames.append(ca_atoms.positions) # append the positions of CA atoms to frames
    frames = np.asarray(frames)

    cluster = CG_cluster() # initialize CG_cluster
    cluster.gamma = 100 # set gamma to 100
    cluster.frames = frames # set frames
    cluster.frame_num = frames.shape[0] # set frame_num
    cluster.atom_num = frames.shape[1] # set atom_num
    cluster.all_atom_num = frames.shape[1] # set all_atom_num
    cluster.eq_positions = np.mean(frames, axis=0) # set eq_positions
    cluster.fluctuation_traj = cluster.frames - np.asarray([cluster.eq_positions for k in range(cluster.frame_num)]) # set fluctuation_traj
    cluster.calculate_rmsf() # calculate rmsf

    all_chi_squares = [] # initialize all_chi_squares and store the temporary chi_square for each trial
    
    #####start editing#####
    num = 0
    if segid == 'A': # set #CG bead to 21 for chain A and 20 for chain B
        num = 21
    elif segid == 'B':
        num = 20
    #####end editing#####

    for k in range(25): # run 25 trials for each chain
        print("Trial %d" % (k+1))
        cluster.initialize_labels(num=num) # initialize labels with num clusters
        cluster.optimize_chi_square(cluster_converge_steps=100, max_steps=100000, output_interval=0, filename="%s_%s%02d" % (num, segid, k+1)) # optimize chi_square
        # cluster_converge_steps: the number of steps for convergence
        # max_steps: the maximum number of steps
        # output_interval: the interval of outputing the results
        # filename: the name of output file
        all_chi_squares.append(cluster.chi_square) # append chi_square to all_chi_squares
        
    best_index = np.argmin(all_chi_squares) # find the index of the best chi_square
    print("Calculation for segid %s complete. Best index is %d " % (segid, best_index + 1))

Calculating RMSF list for all atoms:
Trial 1
Initial optimization complete. Steps = 35, Chi_square = 3552.4025
Second phase optimization complete. Steps = 49083, Chi_square = 1460.1468
Trial 2
Initial optimization complete. Steps = 50, Chi_square = 4626.1219
Second phase optimization complete. Steps = 59016, Chi_square = 1863.4337
Trial 3
Initial optimization complete. Steps = 49, Chi_square = 3616.5150
Second phase optimization complete. Steps = 78214, Chi_square = 1657.1408
Trial 4
Initial optimization complete. Steps = 48, Chi_square = 4081.6248
Second phase optimization complete. Steps = 28351, Chi_square = 1394.1220
Trial 5
Initial optimization complete. Steps = 37, Chi_square = 4354.8773
Second phase optimization complete. Steps = 22934, Chi_square = 1808.4347
Trial 6
Initial optimization complete. Steps = 40, Chi_square = 3521.7800
Second phase optimization complete. Steps = 51403, Chi_square = 1431.2787
Trial 7
Initial optimization complete. Steps = 36, Chi_square = 3782.2220
S

/home/wjb/.conda/envs/openmscg/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wjb/.conda/envs/openmscg/lib/python3.8/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Second phase optimization complete. Steps = 41012, Chi_square = 1616.3689
Trial 2
Initial optimization complete. Steps = 35, Chi_square = 2481.4036
Second phase optimization complete. Steps = 19821, Chi_square = 1240.0256
Trial 3
Initial optimization complete. Steps = 39, Chi_square = 4632.7238
Second phase optimization complete. Steps = 21995, Chi_square = 1219.2864
Trial 4
Initial optimization complete. Steps = 28, Chi_square = 2928.0949
Second phase optimization complete. Steps = 27888, Chi_square = 1908.7379
Trial 5
Initial optimization complete. Steps = 28, Chi_square = 4467.7720
Second phase optimization complete. Steps = 32634, Chi_square = 1215.6347
Trial 6
Initial optimization complete. Steps = 45, Chi_square = 2952.8778
Second phase optimization complete. Steps = 24214, Chi_square = 1608.6076
Trial 7
Initial optimization complete. Steps = 34, Chi_square = 3221.6694
Second phase optimization complete. Steps = 33476, Chi_square = 1074.8929
Trial 8
Initial optimization complete.

In [ ]:
####best results:
####A:23 B:17

In [3]:
##### find the best cgmapping for chain A/B
#####start editing#####
best_ids = [23, 17]
#####end editing#####
for segid in "AB":
    i = "AB".index(segid)
    ca_atoms = system_info.atoms.select_atoms("segid %s and name CA" % segid)
    frames = []
    for frame in system_info.trajectory:
        frames.append(ca_atoms.positions)
    frames = np.asarray(frames)

    cluster = CG_cluster()
    cluster.gamma = 100
    cluster.frames = frames
    cluster.frame_num = frames.shape[0]
    cluster.atom_num = frames.shape[1]
    cluster.all_atom_num = frames.shape[1]
    cluster.eq_positions = np.mean(frames, axis=0)
    cluster.fluctuation_traj = cluster.frames - np.asarray([cluster.eq_positions for k in range(cluster.frame_num)])
    cluster.calculate_rmsf()

    #####start editing#####
    num = 0
    if segid == 'A':
        num = 21
    elif segid == 'B':
        num = 20
    #####end editing#####
    
    cluster.read_labels_from_file("result_%s_%s%02d.txt" % (num, segid, best_ids[i]))
    #cluster.plot_labels(filename="%s%02d" % (segid, best_ids[i]))
    #cluster.visualize_labels(parmfile="../../../ref-trajs/rep1/md_3.part011_System.pdb", trajfile="../../../ref-trajs/rep1/aligned_to_first_frame_system.xtc")

Calculating RMSF list for all atoms:
Calculating RMSF list for all atoms:


## Generate the overall CG map for Chain A and Chain B

In [ ]:
# offset the residue index of the chains after chain A
starter = 0
starters = [0]
for segid in 'AB':
    ca_atoms = system_info.atoms.select_atoms("segid %s and name CA" % segid)
    starter += len(ca_atoms)
    print(len(ca_atoms))
    starters.append(starter)
starters

In [10]:
#####start#####
resolution = 20
label_nums = [23,17] #best result index
nums = [21,20] #CG resolution, #chain A = 21, #chain B = 20
#####end#####
segids = "AB"
mapping_labels = []
for num, segid, label_num in zip(nums, segids, label_nums):
    i = segids.index(segid)
    cgfile = "result_%d_%s%02d.txt" % (num, segid, label_num)
    with open(cgfile) as f:
        aa = f.readlines()
        for line in aa:
            try:
                lst = eval(line)
                lst = list(np.array(lst)+starters[i])
                lst[0]
                new_lst = []
                for item in lst:
                    if type(item) not in (int, np.int64):
                        new_lst.extend(list(range(item[0], item[1]+1)))
                    else:
                        new_lst = lst      
                mapping_labels.append(new_lst)
            except:
                pass

with open("result_overall.txt", 'w') as f:
    for k in range(len(mapping_labels)):
        print(k, file=f)
        print(mapping_labels[k], file=f)

In [51]:
##type in the terminal to generate the yaml file, the AA-mapped CG trajectory (reference trajectory), henm, and lammps input files
####start
python CG_mapping.py -top="md.pdb" -crd="md.trr" -cg="result_overall.txt" -o="arp23"
####end